# Lab E.1: PyTorch Geometric Setup

**Module:** E - Graph Neural Networks  
**Time:** 1 hour  
**Difficulty:** ⭐⭐ (Intermediate)

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- [ ] Understand why graphs are a powerful data structure for AI
- [ ] Install and configure PyTorch Geometric on DGX Spark
- [ ] Load and explore the Cora citation network dataset
- [ ] Visualize graph structure using NetworkX
- [ ] Understand PyG Data objects and their components
- [ ] Create train/validation/test splits for graph data

---

## 📚 Prerequisites

- Completed: Module 1.5 (Neural Networks Fundamentals)
- Completed: Module 2.1 (PyTorch Essentials)
- Knowledge of: Basic Python, NumPy, PyTorch tensors

---

## 🌍 Real-World Context

**Why do we need Graph Neural Networks?**

Not all data fits into neat grids (like images) or sequences (like text). Consider:

| Domain | Nodes | Edges | Task |
|--------|-------|-------|------|
| Social Networks | People | Friendships | Recommend new friends |
| Drug Discovery | Atoms | Chemical bonds | Predict molecule properties |
| Citation Networks | Papers | Citations | Classify paper topics |
| Fraud Detection | Accounts | Transactions | Identify suspicious patterns |
| Knowledge Graphs | Entities | Relationships | Answer complex questions |

**Industry Impact:**
- **Pinterest** uses GNNs for visual recommendations (PinSage)
- **Google Maps** uses GNNs for ETA prediction
- **Drug companies** use GNNs to discover new medicines (AlphaFold)
- **Financial institutions** use GNNs to detect fraud rings

---

## 🧒 ELI5: What Are Graphs?

> **Imagine you're at a birthday party** with all your friends. Now imagine drawing a picture where:
> - Each person is a **dot** (we call these *nodes*)
> - Each friendship is a **line connecting dots** (we call these *edges*)
>
> Now you have a **graph**! It shows who knows who.
>
> Here's the cool part: If you want to predict who might become friends next, you can look at people who have **many friends in common**. Two people who both know Alice, Bob, and Charlie will probably become friends too!
>
> **Graph Neural Networks do exactly this** - they learn patterns from connections to make predictions. To understand a person (node), look at who they're connected to (neighbors). Then look at neighbors of neighbors. This creates a rich understanding of someone's "position" in the social network.
>
> **In AI terms:** GNNs use *message passing* - each node sends messages to its neighbors, collects messages from neighbors, and updates its understanding. After a few rounds of this "telephone game," every node knows about its local neighborhood.

---

## Part 1: Environment Setup

### 1.1 Installing PyTorch Geometric

PyTorch Geometric (PyG) is the most popular library for Graph Neural Networks. It provides:
- Efficient graph data structures
- Pre-built GNN layers (GCN, GAT, GraphSAGE, etc.)
- Popular graph datasets
- Mini-batch training utilities for large graphs

In [ ]:
# First, let's check our environment
import sys
print(f"Python version: {sys.version}")

import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Install PyTorch Geometric and dependencies
# This works on DGX Spark (ARM64 + CUDA)
!pip install -q torch-geometric
!pip install -q networkx matplotlib

In [ ]:
# Verify installation
import torch_geometric
print(f"PyTorch Geometric version: {torch_geometric.__version__}")
print("✅ PyG installed successfully!")

### 🔍 What Just Happened?

We installed:
1. **torch-geometric**: The main PyG library with GNN implementations
2. **networkx**: For graph visualization and analysis
3. **matplotlib**: For plotting

> **DGX Spark Note:** PyG works great on our ARM64 architecture! The library auto-detects CUDA and uses GPU acceleration.

---

## Part 2: Understanding Graph Data Structures

### 2.1 How Computers Represent Graphs

There are two main ways to store a graph:

**1. Adjacency Matrix** - A square matrix where entry (i,j) = 1 if nodes i and j are connected
```
     A  B  C  D
A [[ 0, 1, 1, 0 ],
B  [ 1, 0, 1, 1 ],
C  [ 1, 1, 0, 0 ],
D  [ 0, 1, 0, 0 ]]
```
- ✅ Easy matrix operations
- ❌ Wastes memory for sparse graphs (most real graphs are sparse!)

**2. Edge List** - Just list the connections
```
[[A, B], [A, C], [B, C], [B, D]]
```
- ✅ Memory efficient for sparse graphs
- ✅ PyG uses this format!

Let's see this in action:

In [ ]:
import torch
import numpy as np

# Create a simple graph: A triangle (A-B-C) with D connected to B
# Node indices: A=0, B=1, C=2, D=3

# Method 1: Adjacency Matrix (dense)
adj_matrix = torch.tensor([
    [0, 1, 1, 0],  # A connects to B, C
    [1, 0, 1, 1],  # B connects to A, C, D
    [1, 1, 0, 0],  # C connects to A, B
    [0, 1, 0, 0],  # D connects to B
], dtype=torch.float)

print("Adjacency Matrix (dense):")
print(adj_matrix)
print(f"Memory: {adj_matrix.numel() * 4} bytes\n")

# Method 2: Edge Index (sparse) - PyG format
# Format: [[source nodes], [target nodes]]
edge_index = torch.tensor([
    [0, 0, 1, 1, 1, 2, 2, 3],  # source nodes
    [1, 2, 0, 2, 3, 0, 1, 1],  # target nodes
], dtype=torch.long)

print("Edge Index (sparse - PyG format):")
print(edge_index)
print(f"Memory: {edge_index.numel() * 8} bytes")
print(f"\nEdges: {edge_index.shape[1]}")

### 🧒 ELI5: Edge Index Format

> Think of it like a phone book for friendships:
> - First row: Who's calling (source)
> - Second row: Who's receiving (target)
>
> So `[[0, 1], [1, 2]]` means: "0 calls 1" and "1 calls 2"
>
> For **undirected** graphs (like friendships), we list both directions:
> "0 calls 1" AND "1 calls 0" (both are friends with each other)

In [ ]:
# Visualize our simple graph
import networkx as nx
import matplotlib.pyplot as plt

# Create NetworkX graph from edge index
G = nx.Graph()
G.add_nodes_from(['A', 'B', 'C', 'D'])
G.add_edges_from([('A', 'B'), ('A', 'C'), ('B', 'C'), ('B', 'D')])

# Draw the graph
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G, seed=42)
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=2000, font_size=20, font_weight='bold',
        edge_color='gray', width=2)
plt.title("Our Simple Graph: A Triangle + One Extra Node", fontsize=14)
plt.tight_layout()
plt.show()

print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")
print(f"Average degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}")

---

## Part 3: The Cora Citation Network

### 3.1 What is Cora?

Cora is the "MNIST of graph learning" - a classic benchmark dataset.

- **Nodes**: 2,708 scientific papers
- **Edges**: 10,556 citation links (paper A cites paper B)
- **Features**: 1,433-dimensional bag-of-words for each paper
- **Labels**: 7 categories (Neural Networks, Case-Based, etc.)
- **Task**: Predict the category of each paper based on its content AND citation network

Why is this interesting? A paper's topic is revealed not just by its words, but by **what papers cite it and what papers it cites**. GNNs can leverage this network structure!

In [ ]:
from torch_geometric.datasets import Planetoid
import os

# Download and load Cora dataset
# The Planetoid class handles downloading and preprocessing
dataset = Planetoid(root='/tmp/Cora', name='Cora')

print("=" * 50)
print("CORA DATASET OVERVIEW")
print("=" * 50)
print(f"Number of graphs: {len(dataset)}")
print(f"Number of classes: {dataset.num_classes}")
print(f"Number of node features: {dataset.num_features}")

In [ ]:
# Get the single graph in the dataset
data = dataset[0]

print("\n" + "=" * 50)
print("GRAPH STATISTICS")
print("=" * 50)
print(f"Number of nodes: {data.num_nodes:,}")
print(f"Number of edges: {data.num_edges:,}")
print(f"Average node degree: {data.num_edges / data.num_nodes:.2f}")
print(f"Has isolated nodes: {data.has_isolated_nodes()}")
print(f"Has self-loops: {data.has_self_loops()}")
print(f"Is undirected: {data.is_undirected()}")

### 3.2 Exploring the PyG Data Object

PyG uses a special `Data` object to store graphs. Let's examine its components:

In [ ]:
print("\n" + "=" * 50)
print("DATA OBJECT STRUCTURE")
print("=" * 50)
print(f"\nData object: {data}")
print("\nAvailable attributes:")
for key in data.keys():
    print(f"  - {key}: {data[key].shape if hasattr(data[key], 'shape') else type(data[key]).__name__}")

In [ ]:
# Let's examine each component in detail

print("\n📊 NODE FEATURES (data.x)")
print("-" * 40)
print(f"Shape: {data.x.shape}")
print(f"Dtype: {data.x.dtype}")
print(f"Description: {data.num_nodes} papers, each with {data.num_features} word features")
print(f"Example (first 10 features of first paper): {data.x[0, :10]}")
print(f"Non-zero features in first paper: {(data.x[0] > 0).sum().item()}")

print("\n🔗 EDGE INDEX (data.edge_index)")
print("-" * 40)
print(f"Shape: {data.edge_index.shape}")
print(f"Dtype: {data.edge_index.dtype}")
print(f"Description: [2, num_edges] - source and target node indices")
print(f"First 5 edges:")
for i in range(5):
    src, dst = data.edge_index[0, i].item(), data.edge_index[1, i].item()
    print(f"  Paper {src} → cites → Paper {dst}")

print("\n🏷️ LABELS (data.y)")
print("-" * 40)
print(f"Shape: {data.y.shape}")
print(f"Dtype: {data.y.dtype}")
print(f"Number of classes: {data.y.max().item() + 1}")
print(f"Class distribution:")
for c in range(dataset.num_classes):
    count = (data.y == c).sum().item()
    print(f"  Class {c}: {count} papers ({100*count/data.num_nodes:.1f}%)")

In [ ]:
# Examine the train/val/test masks
print("\n🎭 DATA SPLITS (Masks)")
print("-" * 40)

print(f"Training mask shape: {data.train_mask.shape}")
print(f"Training nodes: {data.train_mask.sum().item()} ({100*data.train_mask.sum().item()/data.num_nodes:.1f}%)")
print(f"Validation nodes: {data.val_mask.sum().item()} ({100*data.val_mask.sum().item()/data.num_nodes:.1f}%)")
print(f"Test nodes: {data.test_mask.sum().item()} ({100*data.test_mask.sum().item()/data.num_nodes:.1f}%)")

# Show how masks work
print("\n📝 How masks work:")
print(f"First 10 mask values: {data.train_mask[:10]}")
print(f"\nTo get training labels: data.y[data.train_mask]")
print(f"Training labels shape: {data.y[data.train_mask].shape}")

### 🔍 What Just Happened?

We explored the Cora dataset structure:

| Component | Shape | Description |
|-----------|-------|-------------|
| `data.x` | [2708, 1433] | Node features (bag-of-words) |
| `data.edge_index` | [2, 10556] | Edge connections (citations) |
| `data.y` | [2708] | Node labels (7 classes) |
| `data.train_mask` | [2708] | Boolean mask for training nodes |
| `data.val_mask` | [2708] | Boolean mask for validation nodes |
| `data.test_mask` | [2708] | Boolean mask for test nodes |

> **Key Insight:** The standard Cora split uses only **140 training nodes** (20 per class) out of 2,708 total! This is a *semi-supervised* setting - we predict labels for nodes using very few labeled examples, leveraging the graph structure.

---

## Part 4: Visualizing the Citation Network

Let's create some visualizations to understand the graph structure.

In [ ]:
from torch_geometric.utils import to_networkx
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Convert to NetworkX for visualization
# Note: We convert to undirected for cleaner visualization
G = to_networkx(data, to_undirected=True)

print(f"NetworkX graph created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")

In [ ]:
# Visualize a small subgraph (full graph is too large to display clearly)
# Let's take nodes 0-100 and their connections

subgraph_nodes = list(range(100))
subgraph = G.subgraph(subgraph_nodes)

# Get labels for coloring
node_colors = data.y[subgraph_nodes].numpy()

# Create a nice color map
class_names = ['Case-Based', 'Genetic Alg.', 'Neural Nets', 'Probabilistic', 
               'Reinfortic', 'Rule Learn.', 'Theory']
cmap = plt.cm.Set3

plt.figure(figsize=(14, 10))
pos = nx.spring_layout(subgraph, seed=42, k=2)
nx.draw(subgraph, pos, 
        node_color=node_colors, 
        cmap=cmap,
        node_size=100, 
        with_labels=False,
        edge_color='lightgray',
        alpha=0.8,
        width=0.5)

# Add legend
handles = [mpatches.Patch(color=cmap(i/6), label=class_names[i]) for i in range(7)]
plt.legend(handles=handles, loc='upper left', fontsize=10)
plt.title("Cora Subgraph: First 100 Papers (Colored by Topic)", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Analyze graph statistics
import numpy as np

print("\n📈 GRAPH STATISTICS")
print("=" * 50)

# Degree distribution
degrees = [d for n, d in G.degree()]
print(f"\nDegree Statistics:")
print(f"  Min degree: {min(degrees)}")
print(f"  Max degree: {max(degrees)}")
print(f"  Mean degree: {np.mean(degrees):.2f}")
print(f"  Median degree: {np.median(degrees):.1f}")

# Clustering coefficient
avg_clustering = nx.average_clustering(G)
print(f"\nClustering coefficient: {avg_clustering:.4f}")
print("  (How likely neighbors of a node are also connected)")

# Connected components
n_components = nx.number_connected_components(G)
print(f"\nNumber of connected components: {n_components}")

# Plot degree distribution
plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.hist(degrees, bins=50, color='steelblue', edgecolor='white')
plt.xlabel('Degree')
plt.ylabel('Count')
plt.title('Degree Distribution')

plt.subplot(1, 2, 2)
plt.hist(degrees, bins=50, color='steelblue', edgecolor='white', log=True)
plt.xlabel('Degree')
plt.ylabel('Count (log scale)')
plt.title('Degree Distribution (Log Scale)')

plt.tight_layout()
plt.show()

print("\n💡 Notice the long tail - a few papers are highly cited (hub nodes)!")

In [ ]:
# Find the most connected papers
degree_dict = dict(G.degree())
sorted_nodes = sorted(degree_dict.items(), key=lambda x: x[1], reverse=True)

print("\n🌟 TOP 10 MOST CITED PAPERS")
print("-" * 40)
for node_id, degree in sorted_nodes[:10]:
    label = data.y[node_id].item()
    print(f"Paper {node_id}: {degree} citations (Class: {class_names[label]})")

### 🧒 ELI5: What Do These Statistics Mean?

> **Degree** = Number of friends. Most papers have few citations, but some "famous" papers have many.
>
> **Clustering Coefficient** = If your friends are also friends with each other. In Cora, it's moderate - papers in the same field tend to cite each other.
>
> **Connected Component** = A group where everyone can reach everyone through some path. Cora is mostly one big component - science is connected!
>
> **Power Law Distribution** = A few papers get LOTS of citations (famous works), while most papers get only a few. This is typical in real networks!

---

## Part 5: Working with PyG Data Objects

### 5.1 Moving Data to GPU

On DGX Spark, we have 128GB of unified memory - plenty of room for graph data!

In [ ]:
# Check memory before loading
if torch.cuda.is_available():
    print(f"GPU Memory before: {torch.cuda.memory_allocated() / 1e6:.1f} MB allocated")

# Move data to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

print(f"\nData moved to: {device}")
print(f"Node features device: {data.x.device}")
print(f"Edge index device: {data.edge_index.device}")

if torch.cuda.is_available():
    print(f"\nGPU Memory after: {torch.cuda.memory_allocated() / 1e6:.1f} MB allocated")

In [ ]:
# Memory usage breakdown
def calculate_memory(tensor):
    """Calculate memory usage of a tensor in MB."""
    return tensor.numel() * tensor.element_size() / 1e6

print("\n💾 MEMORY USAGE BREAKDOWN")
print("-" * 40)
print(f"Node features (x): {calculate_memory(data.x):.2f} MB")
print(f"Edge index: {calculate_memory(data.edge_index):.2f} MB")
print(f"Labels (y): {calculate_memory(data.y):.4f} MB")

total_mb = calculate_memory(data.x) + calculate_memory(data.edge_index) + calculate_memory(data.y)
print(f"\nTotal: {total_mb:.2f} MB")
print(f"\n✅ Tiny compared to our 128GB unified memory!")

### 5.2 Creating Custom Data Splits

In [ ]:
# The default Cora split is very small (140 training nodes)
# Let's create a custom split with more training data

def create_custom_split(data, train_ratio=0.6, val_ratio=0.2, seed=42):
    """
    Create custom train/val/test split for node classification.
    
    Args:
        data: PyG Data object
        train_ratio: Fraction of nodes for training
        val_ratio: Fraction of nodes for validation
        seed: Random seed for reproducibility
    
    Returns:
        Modified data with new masks
    """
    torch.manual_seed(seed)
    
    n_nodes = data.num_nodes
    n_train = int(n_nodes * train_ratio)
    n_val = int(n_nodes * val_ratio)
    
    # Shuffle node indices
    perm = torch.randperm(n_nodes)
    
    # Create masks
    train_mask = torch.zeros(n_nodes, dtype=torch.bool)
    val_mask = torch.zeros(n_nodes, dtype=torch.bool)
    test_mask = torch.zeros(n_nodes, dtype=torch.bool)
    
    train_mask[perm[:n_train]] = True
    val_mask[perm[n_train:n_train + n_val]] = True
    test_mask[perm[n_train + n_val:]] = True
    
    # Update data object
    data.train_mask = train_mask.to(data.x.device)
    data.val_mask = val_mask.to(data.x.device)
    data.test_mask = test_mask.to(data.x.device)
    
    return data

# Create 60/20/20 split
data_custom = create_custom_split(data.clone(), train_ratio=0.6, val_ratio=0.2)

print("Custom Split (60/20/20):")
print(f"  Training: {data_custom.train_mask.sum().item()} nodes")
print(f"  Validation: {data_custom.val_mask.sum().item()} nodes")
print(f"  Test: {data_custom.test_mask.sum().item()} nodes")

---

## ✋ Try It Yourself: Exercise 1

**Task:** Load and explore the **CiteSeer** dataset (another citation network).

1. Load CiteSeer using `Planetoid(root='/tmp/CiteSeer', name='CiteSeer')`
2. Print the number of nodes, edges, features, and classes
3. Compare CiteSeer to Cora - which is larger?
4. Visualize a subgraph of 50 nodes

In [ ]:
# Your code here!
# Hint: The code is almost identical to what we did for Cora

# Step 1: Load CiteSeer
# citeseer = Planetoid(root='/tmp/CiteSeer', name='CiteSeer')

# Step 2: Print statistics
# ...

# Step 3: Compare to Cora
# ...

# Step 4: Visualize subgraph
# ...

<details>
<summary>💡 Hint</summary>

```python
# Load CiteSeer
citeseer = Planetoid(root='/tmp/CiteSeer', name='CiteSeer')
cs_data = citeseer[0]

# Compare
print(f"CiteSeer: {cs_data.num_nodes} nodes, {cs_data.num_edges} edges")
print(f"Cora: {data.num_nodes} nodes, {data.num_edges} edges")
```
</details>

---

## ✋ Try It Yourself: Exercise 2

**Task:** Analyze the neighborhood of a specific node.

1. Pick node 0 (or any node you like)
2. Find all its neighbors (directly connected nodes)
3. What are the labels of the neighbors? Are they mostly the same class?
4. This pattern (similar labels for neighbors) is called **homophily** - crucial for GNNs!

In [ ]:
# Your code here!
# Hint: Use edge_index to find neighbors

# Step 1: Pick a node
target_node = 0

# Step 2: Find neighbors (nodes that have an edge to/from target_node)
# edge_index[0] = source nodes, edge_index[1] = target nodes
# ...

# Step 3: Get labels of neighbors
# ...

# Step 4: Calculate homophily for this node
# ...

<details>
<summary>💡 Hint</summary>

```python
# Move data to CPU for easier manipulation
edge_index_cpu = data.edge_index.cpu()
labels_cpu = data.y.cpu()

# Find neighbors (where target_node is the source)
mask = edge_index_cpu[0] == target_node
neighbors = edge_index_cpu[1][mask]

# Get labels
target_label = labels_cpu[target_node]
neighbor_labels = labels_cpu[neighbors]

# Calculate homophily
same_label = (neighbor_labels == target_label).sum()
homophily = same_label / len(neighbors)
```
</details>

---

## ⚠️ Common Mistakes

### Mistake 1: Forgetting edge direction
```python
# ❌ Wrong: Only checking one direction
neighbors = edge_index[1][edge_index[0] == node]

# ✅ Right: For undirected graphs, check both directions
out_neighbors = edge_index[1][edge_index[0] == node]
in_neighbors = edge_index[0][edge_index[1] == node]
all_neighbors = torch.unique(torch.cat([out_neighbors, in_neighbors]))
```
**Why:** In undirected graphs stored as directed edges, each edge appears twice.

### Mistake 2: Using dense operations on large graphs
```python
# ❌ Wrong: Creates huge dense matrix
adj_matrix = torch.zeros(num_nodes, num_nodes)
for i, j in edge_index.t():
    adj_matrix[i, j] = 1

# ✅ Right: Use sparse operations
from torch_geometric.utils import to_dense_adj
adj_matrix = to_dense_adj(edge_index)  # Only if you really need it!
# Better: Work with edge_index directly
```
**Why:** A graph with 1M nodes would need 1M×1M = 1TB for a dense matrix!

### Mistake 3: Not using masks correctly
```python
# ❌ Wrong: Training on all data
loss = criterion(model(data.x, data.edge_index), data.y)

# ✅ Right: Only train on labeled training nodes
out = model(data.x, data.edge_index)
loss = criterion(out[data.train_mask], data.y[data.train_mask])
```
**Why:** Test nodes should never be used during training - that's cheating!

---

## 🎉 Checkpoint

You've learned:
- ✅ Why graphs are important (social networks, molecules, citations)
- ✅ How graphs are represented (edge index vs adjacency matrix)
- ✅ How to install and use PyTorch Geometric
- ✅ The structure of PyG Data objects (x, edge_index, y, masks)
- ✅ How to visualize graphs with NetworkX
- ✅ Key graph statistics (degree, clustering, connectivity)

---

## 🚀 Challenge (Optional)

**Advanced Challenge:** Calculate the **graph-wide homophily** score.

Homophily = fraction of edges that connect nodes of the same class.

- High homophily (>0.7): Similar nodes are connected → GNNs work great!
- Low homophily (<0.3): Dissimilar nodes are connected → Need special architectures

Calculate homophily for Cora. Is it high or low? What does this mean for GNN performance?

In [ ]:
# Advanced Challenge: Calculate homophily
# Hint: For each edge (i, j), check if y[i] == y[j]

# Your code here!


---

## 📖 Further Reading

- [PyTorch Geometric Documentation](https://pytorch-geometric.readthedocs.io/) - Official docs
- [CS224W: Machine Learning with Graphs](http://web.stanford.edu/class/cs224w/) - Stanford course
- [A Gentle Introduction to GNNs](https://distill.pub/2021/gnn-intro/) - Distill article
- [Graph Representation Learning Book](https://www.cs.mcgill.ca/~wlh/grl_book/) - Free online textbook

---

## 🧹 Cleanup

In [ ]:
# Clear GPU memory
import gc

# Delete large objects
del G, subgraph

# Clear cache
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"GPU memory after cleanup: {torch.cuda.memory_allocated() / 1e6:.1f} MB")

print("✅ Cleanup complete!")

---

## ⏭️ Next Steps

Now that you understand graph data and PyG basics, you're ready to build your first Graph Neural Network!

**In Lab E.2: GCN from Scratch**, you'll:
- Implement the Graph Convolutional Network layer
- Understand the message passing paradigm
- Train a GCN to classify papers in Cora
- Achieve >80% accuracy with just a few lines of code!

Let's go! 🚀